# Welcome to Folium
Folium is a Python library that functions as an intermediary to the Javascript Leaflet library, which does the actual work of rendering map elements in a browser space. This is not relevant most of the time, since the Python library provides pretty robust functionality without the need to know any Javascript. There are some exceptions, but let's ignore those.

This tutorial should take you further than most "Learn Folium tutorials" you will find on, say, Medium. But it won't be comprehensive. Some elements, like the interaction with GeoJSON, can get much more intensive and I touch on them only lightly here.

# Basics - Libraries etc.

In [ ]:
import folium
import pandas as pd
import numpy as np
import re

# By default pandas will cut off the number of rows and columns it will show. This overrides that.
pd.options.display.max_rows = 500000
pd.options.display.max_columns = 600

# Initializing a map, basemaps, etc.

In [ ]:
lon, lat = 34.052235, -118.243683

zoom_start = 10

In [ ]:
m = folium.Map(location=(lon, lat), zoom_start = zoom_start, tiles="OpenStreetMap")

m

Tiles beyond the default ones can be found here: https://leaflet-extras.github.io/leaflet-providers/preview/

In many cases, there will be requirements for such details as attribution, which take the form below.

In [ ]:
folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
                name='Esri.WorldGrayCanvas',
                attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ').add_to(m)

m

# Mapping some points

In [ ]:
folium.Marker(
    location=[34.052235, -118.243683],
    tooltip="This is where the map is centered",
    popup="Downtow LA",
    icon=folium.Icon(color="red"),
).add_to(m)

folium.Marker(
    location=[34.077485887834406, -118.47325409771587],
    tooltip="This is where we work!",
    popup="Getty Center",
    icon=folium.Icon(icon="cloud"),
).add_to(m)

m

## The Map of the Stars!
We can add a bunch of places fairly easily using the pointer function now. Let's try to create an ad hoc star map using the following online list. I've already provided a couple of sites of interest below.
https://winetravelandsong.com/music-history-landmarks/los-angeles-rock-n-roll-landmarks/

Let's do it as a function that replicates the code we ran separately above.

In [ ]:
def pointer(name, tooltip, lat, long):
    folium.Marker(
    location=[lat, long],
    tooltip=tooltip,
    popup=name,
    icon=folium.Icon(color='purple')).add_to(m)

In [ ]:
pointer("The Viper Room", "Where River Phoenix died! :-{", 34.09057643507024, -118.38470184636306)
pointer("The Beverly Hills Hilton", "Not sure what happened here, but it appears on a lot of lists", 34.08194673339452, -118.4133708134921)
m

In [ ]:
trail_coordinates = [
    (34.09057643507024, -118.38470184636306),
    (34.08194673339452, -118.4133708134921),
]

folium.PolyLine(trail_coordinates, tooltip="Coast", color='blue').add_to(m)
m

In [ ]:
# New entries here






# Working with Data
Ok, so let's actually work with some data. I've downloaded some data on traffic incidents from the City of Los Angeles' data portal. 

https://data.lacity.org/Public-Safety/Traffic-Collision-Data-01Jan2015-31May2017-VistaDe/6tkk-tfyj/about_data

This records traffic incidents reported to police between 2010 and now. Let's start by trying to just map them.

In [ ]:
## Let's actually import some data to get started with this
collisions = pd.read_csv('Traffic_Collision_Data_01Jan2015-31May2017_VistaDelMar-Imperial-Culver_20250331.csv')

def pointer(name, tooltip, lat, long):
    folium.Marker(
    location=[lat, long],
    tooltip=tooltip,
    popup=name,
    icon=folium.Icon(color='lightgreen')).add_to(m)

In [ ]:
collisions.head(2)

In [ ]:
# The points are encoded in a format called Well Known Text (WKT). There are libraries to work with wkt, albeit it is still inconvenient.
# Let's see how
import shapely.wkt
s = collisions['Location'].iloc[1]
oop = shapely.wkt.loads(s)
print(oop)

In [ ]:
def longitudinizer(pair):
    longitude = re.sub('(.*\s)(\()(.*)(\s)(.*)(\)$)','\\5',pair)
    return longitude

def latitudinizer(pair):
    latitude = re.sub('(.*\s)(\()(.*)(\s)(.*)(\)$)','\\3',pair)
    return latitude

In [ ]:
for i in collisions['Location']:
    print(latitudinizer(i))

In [ ]:
collisions['Latitude'] = collisions['Location'].apply(lambda x: latitudinizer(x))
collisions['Longitude'] = collisions['Location'].apply(lambda x: longitudinizer(x))

In [ ]:
collisions.head(2)

In [ ]:
for code, age, longitude, latitude in zip(collisions['Crime Code Description'], collisions['Victim Age'], collisions['Longitude'].astype(float), collisions['Latitude'].astype(float)):
    pointer(code, age, longitude, latitude)

m

# Working with Layers and Circle Markers
Points are fine. There are ways to substitute your own images for the little default markers. Realistically, I don't love the look of it all... it is very Google Maps-y. The way to go, I think, is to define circle markers of your own and style them. These can be made variable, in terms of size, to reflect some underlying quantity.

Let's also do this a different way, making use of the "layer" feature, that will let us group features together in ways that, for example, allow us to switch them on and off.

To get into this, let's continue to look at LA crime data. This is all incidents from 2020 onward. The file is kind of huge, so let's download it separately and put it into the same directory as this Notebook.

https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8/about_data

In [ ]:
crime = pd.read_csv('Crime_Data_from_2020_to_Present_20250331.csv')

In [ ]:
crime = crime.head(2000)

In [ ]:
crime.head(2)

In [ ]:
m = folium.Map(location=(lon, lat), zoom_start = zoom_start, tiles="CartoDB Positron")
circles = folium.FeatureGroup("Circle Markers").add_to(m)
folium.features.CircleMarker((34.0375, -118.3506), radius=3, color='black', fill=False, fill_color=0.6, weight=0.3).add_to(circles)

m

In [ ]:
#import random
for code, age, latitude, longitude in zip(crime['Crm Cd Desc'], crime['Vict Age'], crime['LAT'].astype(float), crime['LON'].astype(float)):
    label = (code + ' involving someone ' + str(age) + ' years old.')
    folium.features.CircleMarker((latitude, longitude), 
                                 #radius= random.randrange(1, 10), 
                                 radius= 3,
                                 color='black', 
                                 fill=False, 
                                 fill_color=0.6, 
                                 weight=0.3,
                                 popup=folium.Popup(label),
                                ).add_to(circles)

folium.LayerControl().add_to(m)
m

# Working with GeoJSON - variation on the basic mapping function

# Working with GeoJSON - Working with polygons